In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../")

In [2]:
from master import * 

/usr/local/lib/python3.7/dist-packages/torch/utils/tensorboard/__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, '__version__') or LooseVersion(tensorboard.__version__) < LooseVersion('1.15'):
/usr/local/lib/python3.7/dist-packages/torch/utils/tensorboard/__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, '__version__') or LooseVersion(tensorboard.__version__) < LooseVersion('1.15'):


Log path = /root/src/aleister/log/logging.log


In [3]:
model_id = 'test004'

# args=['--symbol','BTC', '--execute_mode',"prepro",
#       '--model_id',model_id,'--model_name','slstm',
#       '--config_source','ini',
#       '--user',"Misaka00001", '--source','ipynb',
#       "--general_config_mode", "DOCKER","--private_api_mode","DEFAULT",
#       '-train_sd','20210101','-train_ed','20210103',
#       '-valid_sd','20210104','-valid_ed','20210105',
#       '-test_sd','20210106','-test_ed','20210107']

# args=['--symbol','BTC', '--execute_mode',"train",
#       '--model_id',model_id,'--model_name','slstm',
#       '--config_source','ini',
#       '--user',"Misaka00001", '--source','ipynb',
#       "--general_config_mode", "DOCKER","--private_api_mode","DEFAULT",
#       '-train_sd','20210101','-train_ed','20210103',
#       '-valid_sd','20210104','-valid_ed','20210105',
#       '-test_sd','20210106','-test_ed','20210107']

# args=['--symbol','BTC', '--execute_mode',"gtrain",
#       '--model_id',model_id,'--model_name','slstm',
#       '--config_source','ini',
#       '--user',"Misaka00001", '--source','ipynb',
#       "--general_config_mode", "DOCKER","--private_api_mode","DEFAULT",
#       '-train_sd','20210101','-train_ed','20210103',
#       '-valid_sd','20210104','-valid_ed','20210105',
#       '-test_sd','20210106','-test_ed','20210107']

# args=['--symbol','BTC', '--execute_mode',"deploy_model",
#       '--model_id',model_id,'--model_name','slstm',
#       '--config_source','ini',
#       '--user',"Misaka00001", '--source','ipynb',
#       "--general_config_mode", "DOCKER","--private_api_mode","DEFAULT",
#       '-train_sd','20210101','-train_ed','20210103',
#       '-valid_sd','20210104','-valid_ed','20210105',
#       '-test_sd','20210106','-test_ed','20210107']


args=['--symbol','BTC', '--execute_mode',"rpredict",
      '--model_id',model_id,'--model_name','slstm',
      '--config_source','ini',
      '--user',"Misaka00001", '--source','ipynb',
      "--general_config_mode", "DOCKER","--private_api_mode","DEFAULT",
      '-train_sd','20210101','-train_ed','20210103',
      '-valid_sd','20210104','-valid_ed','20210105',
      '-test_sd','20210106','-test_ed','20210107']

# args=['--symbol','BTC', '--execute_mode',"train",
#       '--model_id','test002','--model_name','cgm',
#       '--config_source','ini',
#       '--user',"Misaka00001", '--source','ipynb',
#       "--general_config_mode", "DOCKER","--private_api_mode","DEFAULT",
#       '-train_sd','20210101','-train_ed','20210103',
#       '-valid_sd','20210104','-valid_ed','20210105',
#       '-test_sd','20210106','-test_ed','20210107']

In [4]:
parser = make_parser()
arg_dict = vars(parser.parse_args(args))

# meta
_id = arg_dict["model_id"]
general_config_mode = arg_dict["general_config_mode"].upper()
private_api_mode = arg_dict["private_api_mode"].upper()
sym = arg_dict["symbol"]

model_name = arg_dict["model_name"].upper()

om = OperateMaster()
om.load_meta(_id, model_name, sym, general_config_mode, private_api_mode)
om.init_prepro()

# mlflow
mlflow_tags = {
    MLFLOW_USER: arg_dict["user"],
    MLFLOW_SOURCE_NAME: arg_dict["source"],
    MLFLOW_RUN_NAME: f"TRAIN_{dt.now().strftime('%y%m%d%H%M%s')}"
}
mlflow_client_kwargs = {
    "tracking_uri": os.environ['MLFLOW_TRACKING_URI']
}
om.set_mlflow_settings(mlflow_client_kwargs, mlflow_tags)

# load conofigs into each module
om.init_learning()


Log path = /root/src/aleister/log/logging.log
Log path = /root/src/aleister/log/logging.log


In [5]:

if arg_dict["execute_mode"] == "prepro":
    pass
    om.init_dataGen()
    # period
    train_start = arg_dict["train_start_date"] if "train_start_date" in arg_dict.keys() else None
    train_end = arg_dict["train_end_date"] if "train_end_date" in arg_dict.keys() else None
    valid_start = arg_dict["valid_start_date"] if "valid_start_date" in arg_dict.keys() else None
    valid_end = arg_dict["valid_end_date"] if "valid_end_date" in arg_dict.keys() else None
    test_start = arg_dict["test_start_date"] if "test_start_date" in arg_dict.keys() else None
    test_end = arg_dict["test_end_date"] if "test_end_date" in arg_dict.keys() else None
    # om.preprocessing(sym, train_start, train_end, valid_start, valid_end, test_start, test_end)
else:
    if arg_dict["execute_mode"] == "train":
        om.init_learning()
        # om.train()
    elif arg_dict["execute_mode"] == "gtrain":
        om.init_learning()
        # om.gtrain()
    elif arg_dict["execute_mode"] == "deploy_model":
        om.deploy_best_model()
    elif arg_dict["execute_mode"] == "rpredict":
        om.init_dataGen()
        # om.realtime_predict()


Log path = /root/src/aleister/log/logging.log


In [6]:
le = om.le
fp = om.fp
dg = om.dg

## Preprocessing

In [7]:
om.preprocessing(sym, train_start, train_end, valid_start, valid_end, test_start, test_end)

In [11]:
fp._logger.info(f"{train_start}, {train_end}, {valid_start}, {valid_end}, {test_start}, {test_end}")
fetch_start = min([_date for _date in [train_start, train_end, valid_start, valid_end, test_start, test_end] if
                   _date is not None])
fetch_end = max([_date for _date in [train_start, train_end, valid_start, valid_end, test_start, test_end] if
                 _date is not None])
# dg.get_hist_data(ch="trade", sym="BTC", sd="20220101", ed="20220107")

#TODO: outside
fetch_start=train_start
fetch_end=test_end
output_data_structure =fp.model_config.get("OUTPUT_DATA_STRUCTURE")
input_data_structure = fp.model_config.get("INPUT_DATA_STRUCTURE")



In [12]:
output_data_structure

'flatten_v1'

In [15]:
data_tree['X']

,rel_open,rel_high,rel_low,rel_close,rel_size,buy_size_ratio,sell_size_ratio,movingBinary
datetime,,,,,,,,
2021-01-01 06:00:00,-0.135220,-0.135359,-0.135220,-0.135359,-0.391074,1.445578e-06,0.000000e+00,1.0
2021-01-01 06:01:00,-0.134641,-0.135363,-0.134641,-0.135363,-2.098644,2.834467e-08,0.000000e+00,1.0
2021-01-01 06:02:00,-0.136095,-0.136817,-0.137561,-0.138283,0.233795,0.000000e+00,5.332341e-06,1.0
2021-01-01 06:03:00,-0.137557,-0.138275,-0.137558,-0.138275,-0.074774,0.000000e+00,2.620288e-06,1.0
2021-01-01 06:04:00,-0.137553,-0.138269,-0.137553,-0.138269,-2.285730,0.000000e+00,1.612103e-08,1.0
...,...,...,...,...,...,...,...,...
2021-01-08 05:25:00,-0.006339,-0.006820,-0.006791,-0.007083,-0.195825,5.668934e-07,1.486855e-06,1.0
2021-01-08 05:26:00,-0.006106,-0.006828,-0.006719,-0.007441,-0.922552,0.000000e+00,3.720238e-07,1.0
2021-01-08 05:27:00,-0.005713,-0.005532,-0.005713,-0.005532,0.081052,4.258787e-06,2.480159e-08,1.0


In [13]:
data_tree =dg.get_Xy(mode='train',input_data_structure=input_data_structure,output_data_structure=output_data_structure,sym=sym, sd=fetch_start, ed=fetch_end)
fp._logger.info("[DONE] Get prepro raw datas. {0}~{1}".format(fetch_start, fetch_end))

/usr/local/lib/python3.7/dist-packages/pandas/core/groupby/generic.py:1069: ResourceWarning:

unclosed <socket.socket fd=84, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 48732), raddr=('127.0.0.1', 51661)>

/usr/local/lib/python3.7/dist-packages/pandas/core/groupby/generic.py:1069: ResourceWarning:

unclosed <socket.socket fd=85, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 51661), raddr=('127.0.0.1', 48732)>

/usr/local/lib/python3.7/dist-packages/pandas/core/groupby/generic.py:1069: ResourceWarning:

unclosed <socket.socket fd=86, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('172.30.0.34', 43344)>

/usr/local/lib/python3.7/dist-packages/pandas/core/groupby/generic.py:1069: ResourceWarning:

unclosed <socket.socket fd=90, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('172.30.0.34', 43348)>

/usr/local/lib/python3.7/dist-packages/pandas/core/g

In [62]:
# Split train/test/val ineach leaf
term_splited_data_tree = fp.split_child_by_period(data_tree,train_start, train_end, valid_start, valid_end, test_start, test_end)
fp._logger.info("[DONE] Split by term.")

# Make each period tree
train_datas = fp.select_child_by_period(term_splited_data_tree, 'train')
test_datas = fp.select_child_by_period(term_splited_data_tree, 'test')
val_datas = fp.select_child_by_period(term_splited_data_tree, 'valid')
fp._logger.info("[DONE] picked by term.")

# to ndarray objects
train_datas = fp.get_input_structures(input_data_structure)(train_datas)
test_datas = fp.get_input_structures(input_data_structure)(test_datas, scalers=train_datas['scaler'])
val_datas = fp.get_input_structures(input_data_structure)(val_datas, scalers=train_datas['scaler'])
fp._logger.info("[DONE] To numpy and numpy ")

#  Note: If other data except  for X is using. Fix X_train": X_trains, "X_val":X_vals, "X_test": X_tests
fp.save_numpy_datas(**{
    "X_train": train_datas['data']['X'], "X_val": test_datas['data']['X'], "X_test": val_datas['data']['X'],
    "y_train": train_datas['data']['y'], "y_val": test_datas['data']['y'], "y_test": val_datas['data']['y'],
    "X_scaler": train_datas['scaler'],"note":{'input_dim':train_datas['input_dim']}
})

In [65]:
scalers = fp.load_numpy_datas(["X_scaler"])
scaler_list = []
for scaler in scalers:
    scaler_list.append(scaler[0])

Log path = /root/src/aleister/log/logging.log


In [66]:
test_datas = fp.get_input_structures(input_data_structure)(test_datas, scalers=train_datas['scaler'])

[MinMaxScaler()]

In [67]:
train_datas['scaler']

[MinMaxScaler()]

## Train 

In [7]:
obj_keys = ["X_train", "X_val", "y_train", "y_val", "note"]
X_trains, X_vals, y_trains, y_vals, note = fp.load_numpy_datas(obj_keys)
om.train_worker(X_trains, X_vals, y_trains, y_vals,  note )


61


KeyError: 'datasetparams'

In [14]:
X_trains

array([-1.35220309e-01, -1.35358831e-01, -1.35220309e-01, -1.35358831e-01,
       -3.91073550e-01,  1.44557823e-06,  0.00000000e+00,  1.00000000e+00])

In [34]:
predictions

array([[9.99979973e-01, 5.17890703e-06, 1.49594198e-05],
       [9.99979854e-01, 5.17760282e-06, 1.49675225e-05],
       [9.99979854e-01, 5.16807131e-06, 1.50279675e-05],
       ...,
       [1.07656269e-05, 3.91482763e-06, 9.99985337e-01],
       [1.08758677e-05, 3.94127801e-06, 9.99985218e-01],
       [9.99778450e-01, 1.91520576e-05, 2.02424315e-04]], dtype=float32)

## Gtrain

In [15]:
om.gtrain()

0


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3420: RuntimeWarning:

Mean of empty slice.

/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:188: RuntimeWarning:

invalid value encountered in double_scalars

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3420: RuntimeWarning:

Mean of empty slice.

/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:188: RuntimeWarning:

invalid value encountered in double_scalars



ValueError: need at least one array to concatenate

In [22]:
len(param_grid)

12

## train worker 

In [17]:


X_trains, X_vals, y_train, y_val, note = X_trains, X_vals, y_trains, y_vals, note
# train set up
model_params = {_k: le.hparams[_k] for _k in le.hparams["structure_params"]}
model_params['input_dim'] = note['input_dim']

le.get_model_instance(le.model_name, model_params)

fp.get_dataset_fn(le.hparams["dataset"])
dataset_params = {_k: le.hparams[_k] for _k in le.hparams["dataset_params"]}
train_loader, val_loader, _, _ = fp.get_dataloader(le.hparams["dataset"], X_trains, y_train, X_vals,
                                                        y_val, None, None, **dataset_params)

61


In [18]:
lossfn_params = {}
le.get_loss_fn(le.hparams["optimizer"], {})

optim_params = {
    'params': le.model.parameters(),
    'weight_decay': le.hparams["weight_decay"],
    'lr': le.hparams["lr"]
}

le.get_optimizer(le.hparams["loss_fn"], optim_params)

# train
le.train(train_loader, val_loader,
              batch_size=le.hparams["batch_size"], n_epochs=le.hparams["n_epoch"],
              n_features=1)

'8c83641bbc8948f88acc2793f9e22e1a'

## Train breakdown

In [37]:
batch_size=64
n_epochs=10
n_features=1
import numpy as np
import torch
from sklearn.metrics import accuracy_score

In [33]:
le._logger.info("[Start] Training. ID={0}".format(le.id))
# _ = le.get_model_save_path()  #todo : designate path

# setup
le.prediction = {}
le.predictions_out = {}
le.truths = {}
le.truths_out = {}
le.out_class = eval(le.model_config.get("OUT_CLASS"))

# mlflow
dict_config = {
    "batch_size": batch_size,
    "n_epochs": n_epochs,
    "optimizer": le.optimizer_name,
    "loss_fn": le.loss_fn_name
}

le.mlwriter.create_experiment(le.id, le.mlflow_tags)
le.mlwriter.log_params_from_omegaconf_dict(dict_config)

# start
for epoch in range(1, n_epochs + 1):
    batch_losses = []
    le.train_loader = train_loader
    for x_batchs, y_batchs in train_loader:
        le.optimizer.zero_grad()
        loss = le.train_step(x_batchs, y_batchs)
        batch_losses.append(loss)
    training_loss = np.mean(batch_losses)
    le.train_losses.append(training_loss)
    le.mlwriter.log_metric('train_loss', training_loss, epoch)

    with torch.no_grad():
        batch_val_losses = []
        predictions = []
        truths = []
        for x_vals, y_val in val_loader:
            prediction, truth, val_loss = le.eval_step(x_vals, y_val)
            predictions.append(prediction)
            truths.append(truth)
            batch_val_losses.append(val_loss)

        validation_loss = np.mean(batch_val_losses)
        le.val_losses.append(validation_loss)
        
        predictions = np.concatenate(predictions)
        truths = np.concatenate(truths)

        # record
        le.prediction["val"], le.truths["val"] = predictions, truths
        predictions_out, truths_out = le.convert_model_value(predictions), le.convert_model_value(truths)
        le.predictions_out["val"], le.truths_out["val"] = predictions_out, truths_out
        acc = accuracy_score(truths_out, predictions_out)

        le.mlwriter.log_metric('valid_acc', acc, epoch)
        le.mlwriter.log_metric('valid_loss', validation_loss, epoch)
        # print(le.prediction["val"])
        # print(le.predictions_out["val"])

    if (epoch <= 10) | (epoch % 50 == 0):
        le._logger.info(
            f"[{epoch}/{n_epochs}] Training loss: {training_loss:.4f}\t Validation loss: {validation_loss:.4f}"
        )

# record last 
le.save_numpy_obj(predictions, "val_preds.csv")
le.save_numpy_obj(truths, "val_truth.csv")
le.save_numpy_obj(predictions_out, "val_preds_out.csv", type='int')
le.save_numpy_obj(truths_out, "val_truth_out.csv", type='int')

# le.save_model()
le.save_mlflow_model()
le._logger.info("[DONE] Training. ID={0}".format(le.id))


TypeError: forward() missing 2 required positional arguments: 'node_feature' and 'adj'

### Deploy 

In [7]:
om.deploy_best_model()

In [12]:
type({"d":"dd"})==dict

True

### Realtime prediction

In [8]:
uri = om.load_prod_model()
om.le.load_mlflow_model(uri)

In [39]:
om.dg.init_mqclient("realtime")

In [40]:
realtime_datas, note = om.realtime_preprocessing()

# train set up
model_params = {_k: om.le.hparams[_k] for _k in om.le.hparams["structure_params"]}
model_params['input_dim'] = note['input_dim']

om.le.get_model_instance(om.le.model_name, model_params)

om.fp.get_dataset_fn(om.le.hparams["dataset"])
dataset_params = {_k: om.le.hparams[_k] for _k in om.le.hparams["dataset_params"]}
_, _, _, test_loader_one = om.fp.get_dataloader(
    om.le.hparams["dataset"], None, None, None, None,
    realtime_datas["data"]["X"], realtime_datas["data"]["y"], **dataset_params)

prediction = om.le.predict(test_loader_one)

/usr/lib/python3.7/json/decoder.py:353: ResourceWarning:

unclosed <socket.socket fd=88, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 33834), raddr=('127.0.0.1', 36091)>

/usr/lib/python3.7/json/decoder.py:353: ResourceWarning:

unclosed <socket.socket fd=89, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 36091), raddr=('127.0.0.1', 33834)>

/usr/lib/python3.7/json/decoder.py:353: ResourceWarning:

unclosed <socket.socket fd=90, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('172.30.0.34', 36426)>



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33


In [41]:
prediction

array([[0.3415096 , 0.30594435, 0.35254607],
       [0.34151688, 0.30586404, 0.35261902],
       [0.34152302, 0.3057909 , 0.35268608],
       [0.34153852, 0.30565593, 0.3528056 ],
       [0.3415452 , 0.30555186, 0.35290298],
       [0.34153417, 0.3053938 , 0.353072  ],
       [0.34127048, 0.3050306 , 0.35369885],
       [0.34033054, 0.3049156 , 0.3547539 ],
       [0.337725  , 0.3060506 , 0.35622436],
       [0.32880434, 0.3147323 , 0.3564633 ],
       [0.33037305, 0.31286868, 0.35675827],
       [0.32587644, 0.3180891 , 0.35603452],
       [0.328398  , 0.31463355, 0.35696852],
       [0.33168358, 0.31134278, 0.35697368],
       [0.32661828, 0.3174176 , 0.35596406],
       [0.3243381 , 0.3194604 , 0.35620144],
       [0.3264847 , 0.3164663 , 0.35704902],
       [0.32422766, 0.31926897, 0.35650343],
       [0.32788235, 0.3147693 , 0.35734838],
       [0.32715115, 0.31599855, 0.35685027],
       [0.32450613, 0.31907368, 0.3564202 ],
       [0.32345596, 0.31998175, 0.35656226],
       [0.

In [42]:
import numpy as np
import json


In [43]:
om.init_realpred_mq()

In [44]:
om.mq_provider.connect_mq()

In [45]:
om.mq_provider.publish_mq(prediction)

In [7]:
# todo: mode conider
uri = om.load_prod_model()
om.le.load_mlflow_model(uri)

realtime_datas, note = om.realtime_preprocessing()

In [8]:

# train set up
model_params = {_k: om.le.hparams[_k] for _k in om.le.hparams["structure_params"]}
model_params['input_dim'] = note['input_dim']

om.le.get_model_instance(om.le.model_name, model_params)

om.fp.get_dataset_fn(om.le.hparams["dataset"])
dataset_params = {_k: om.le.hparams[_k] for _k in om.le.hparams["dataset_params"]}

In [9]:
 realtime_datas["data"]["y"]

[None]

In [28]:


_, _, test_loader, test_loader_one = om.fp.get_dataloader(
    om.le.hparams["dataset"], None, None, None,None, 
    realtime_datas["data"]["X"], realtime_datas["data"]["y"], **dataset_params)

prediction = om.le.predict(test_loader_one)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [29]:
prediction

array([[0.3245815 , 0.39931172, 0.27610686],
       [0.32454658, 0.39937896, 0.2760745 ],
       [0.32437223, 0.39966905, 0.27595875],
       [0.3242796 , 0.39981595, 0.27590445],
       [0.3244523 , 0.39959255, 0.2759551 ],
       [0.3243367 , 0.39968795, 0.27597538],
       [0.32373467, 0.39987832, 0.276387  ],
       [0.32428795, 0.3999641 , 0.27574793],
       [0.32314497, 0.3986499 , 0.2782052 ],
       [0.31712675, 0.38409546, 0.2987778 ],
       [0.31790307, 0.3901374 , 0.29195955],
       [0.319827  , 0.39568365, 0.28448936],
       [0.31931797, 0.39416865, 0.28651342],
       [0.3206365 , 0.3969702 , 0.28239328],
       [0.32299575, 0.40116313, 0.27584112],
       [0.3189801 , 0.38995737, 0.29106253],
       [0.32102448, 0.3962355 , 0.28274003],
       [0.31982166, 0.3934258 , 0.28675255],
       [0.32250905, 0.39965972, 0.27783126],
       [0.3213373 , 0.3956371 , 0.28302556],
       [0.3206634 , 0.39435068, 0.28498593]], dtype=float32)

In [ ]:
import torch
data_loader=test_loader
with torch.no_grad():
    predictions = []
    truths = []
    le.test_loader = data_loader
    for x_tests, y_test in data_loader:
        prediction, truth, val_loss = le.eval_step(x_tests, y_test)
        predictions.append(prediction)
        truths.append(truth)

In [81]:
4//5

0

In [15]:
len( test_loader_one)

21

In [38]:
import pandas as pd
for _key in json_rst.keys():
    try:
        df = pd.DataFrame(json_rst[_key])
        print(1)
        df["datetime"] = df["time"].apply(lambda x: dl.strYMDHMSF_to_dt(x))
        del df["time"]
        df.set_index("datetime", inplace=True)
        dfs[_key] = df
        dg._logger.info(f"[DONE] Load data. Key={_key}")
    except Exception as e:
        dg._logger.warning(f"[Failure] Cannot load data. Key={_key}:{e}",exc_info=True)
        

1
1


In [39]:
dfs

{'orderbook':                         symbol      bids0  bids0_size      bids1  bids1_size  \
 datetime                                                                       
 2022-04-01 18:20:13.846    BTC  5524214.0      0.0002  5524200.0       0.001   
 2022-04-01 18:20:14.142    BTC  5524214.0      0.0002  5524200.0       0.001   
 2022-04-01 18:20:14.856    BTC  5524214.0      0.0002  5524200.0       0.001   
 2022-04-01 18:20:15.366    BTC  5524214.0      0.0002  5524200.0       0.001   
 2022-04-01 18:20:15.872    BTC  5524214.0      0.0002  5524200.0       0.001   
 ...                        ...        ...         ...        ...         ...   
 2022-04-01 18:20:59.421    BTC  5524214.0      0.0002  5524200.0       0.001   
 2022-04-01 18:20:59.912    BTC  5524214.0      0.0002  5524200.0       0.001   
 2022-04-01 18:21:00.418    BTC  5524214.0      0.0002  5524200.0       0.001   
 2022-04-01 18:21:00.924    BTC  5524214.0      0.0002  5524200.0       0.001   
 2022-04-01 18:

In [27]:
from util import daylib
from util import utils
dl = daylib.daylib()

In [ ]:
# prepro
X = self.dg.get_Xy(trades=trades, orderbooks=orderbooks, mode="realtime", method='flatten_v1')
X, _ = self.fp.feature_label_split(df=X, target_col=ans_col)
X, _ = self.fp.scalingX(X, scaself.ler)

In [44]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import TensorDataset

class SequenceDataset(Dataset):
    def __init__(self):
        super().__init__(1, 2, 3)
        # super().__init__(batch_size=1, drop_last=False)
        self.X_tensors = tensors[:-1]
        self.y_tensor = tensors[-1]
        self.window_size = params["window_size"]
        self.batch_size = params["batch_size"]  # keep
        self.Xs = [X_tensor.numpy() for X_tensor in self.X_tensors]

        if self.y_tensor is None:
            # Note; Xs[0] must have data length
            length = len(self.Xs[0])
            self.y = np.array([np.nan for _ in range(length)])
        else:
            self.y = self.y_tensor.numpy()

        if self.batch_size > len(self.y):
            self.batch_size = len(self.y)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        print(idx)
        if idx >= (self.window_size - 1):
            idx_start = idx - self.window_size + 1
            Xs = []
            for X in self.Xs:
                Xs.append(X[idx_start:idx + 1:])
        else:
            n_padding = self.window_size - idx - 1
            Xs = []
            for X in self.Xs:
                padding = np.repeat(X[0][None, :], n_padding, axis=0)
                X_origin = X[:idx + 1]
                X_padded = np.vstack([X_origin, padding])
                Xs.append(X_padded)

        ys = self.y[idx]
        # print(ys)
        return Xs, ys


In [45]:

SequenceDataset()

TypeError: object.__init__() takes exactly one argument (the instance to initialize)

In [40]:
class Car:
    def __init__(self, make, model, year):
        self.make = make
        self.model = model
        self.year = year
        self.odometer_readings = 0

    def getname(self):
        longname = f'{self.year} {self.model} {self.year}'
        return longname.title()

    def read_odometer(self):
        print(f'This car has {self.odometer_readings} miles on it')

    def update_odometer(self, mileage):
        if mileage >= self.odometer_readings:
            self.odometer_readings = mileage
        else:
            print('You cant roll back  the odometer')

    def increment_odometer(self, mileage):
        self.odometer_readings += miles

class Battery:
    def __init__(self, battery_size = 75):
        self.battery_size = battery_size

    def describe_battery(self):
        print(f'This car has a {self.battery_size}-kWh battery in it')

    def get_range(self):
        if self.battery_size == 75:
            range = 260
        elif self.battery_size == 1000:
            range = 310
        print(f"This car can go {range} miles on a full charge")

class electriccar(Car):
    def __init__(self, make, model, year):
        super().__init__(make, model, year)
        self.battery = Battery()

In [41]:
electriccar(1,2,3)